# **Setup and Library Imports**

### **Installing Required Packages**

In [21]:
! pip install --quiet "transformers[torch]"
! pip install --quiet evaluate
! pip install --quiet tabulate
! pip install --quiet ipywidgets
! pip install --quiet datasets
! pip install --quiet pillow
! pip install --quiet scikit-learn
! pip install --quiet tensorboard

### **Importing Libraries**

In [22]:
# PyTorch for tensor operations
import torch

# Hugging Face libraries for training and transformer models
from transformers import Trainer, TrainingArguments, AdamW
from transformers import AutoImageProcessor, AutoModelForImageClassification

# Evaluation metrics and utilities
import evaluate
import numpy as np
from datetime import datetime

# Loading datasets for training and evaluation
from datasets import load_dataset

# Data manipulation and display utilities
import pandas as pd
from tabulate import tabulate
from collections import Counter

### **Logging into Hugging Face Hub**

In [23]:
from huggingface_hub import notebook_login

# Execute the login function to access the Hugging Face account
# notebook_login()

### **Connect to google drive**

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Defining Model, Dataset Paths, and Output Directories**

Define paths for saving model training outputs and logs, incorporating model and dataset names along with the current date.

In [25]:
# Define model and dataset paths
model_path = "facebook/dinov2-base"
dataset_path = "cvmil/rice-leaf-disease-augmented-v2"
train_epochs = 15

# Create a timestamped output directory for saving training results
dt_string = datetime.now().strftime("%m%d%y")
# dt_string = "111824"
output_dir = f"./drive/Shareddrives/CS198-Drones/training_output/{model_path.split('/')[-1]}_{dataset_path.split('/')[-1]}_{dt_string}"

# Define directory for storing training logs
logging_dir = f"{output_dir}/logs"

# **Data Preparation and Processing Pipeline**

This section handles the dataset loading, label extraction, image processing setup, and defines necessary functions for data transformation, batching, and metric computation to prepare the data for model training and evaluation.

### **Load Dataset and Extract Labels**

Load the dataset from huggingface and extract the class labels from the training data.

In [26]:
# Load the dataset
dataset = load_dataset(dataset_path)

# Extract class labels from the training set
labels = dataset['train'].features['label'].names

Generate and display a table showing class distribution across training and validation splits.

In [27]:
label_mapping = dataset['train'].features['label'].int2str

# Count the number of samples per class in each split
train_counts = Counter(dataset['train']['label'])
validation_counts = Counter(dataset['validation']['label'])

# Create a DataFrame for the class distribution
data = {
    'ID': list(range(len(labels))),
    'Label': labels,
    'Training': [train_counts[i] if i in train_counts else 0 for i in range(len(labels))],
    'Validation': [validation_counts[i] if i in validation_counts else 0 for i in range(len(labels))],
}

# Display the class distribution in a table format
df = pd.DataFrame(data)
print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

+------+------------------------+------------+--------------+
|   ID | Label                  |   Training |   Validation |
+======+========================+============+==============+
|    0 | Bacterial Leaf Blight  |       2000 |          250 |
+------+------------------------+------------+--------------+
|    1 | Brown Spot             |       2000 |          250 |
+------+------------------------+------------+--------------+
|    2 | Healthy Rice Leaf      |       2000 |          250 |
+------+------------------------+------------+--------------+
|    3 | Leaf Blast             |       2000 |          250 |
+------+------------------------+------------+--------------+
|    4 | Leaf Scald             |       2000 |          250 |
+------+------------------------+------------+--------------+
|    5 | Narrow Brown Leaf Spot |       2000 |          250 |
+------+------------------------+------------+--------------+
|    6 | Rice Hispa             |       2000 |          250 |
+------+

### **Initialize Image Processor**

Load and initialize the image processor from the pre-trained model.

In [28]:
# Load the image processor from the pre-trained model
processor = AutoImageProcessor.from_pretrained(model_path)
print(processor)

BitImageProcessor {
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "BitImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 256
  }
}



### **Data Preparation and Processing Pipeline**

Create mappings for label-to-ID and ID-to-label.

In [29]:
label2id = {c: idx for idx, c in enumerate(labels)}
id2label = {idx: c for idx, c in enumerate(labels)}

Define the transformation function to process the image batch.

In [30]:
def transforms(batch):
    batch['image'] = [x.convert('RGB') for x in batch['image']]
    inputs = processor(batch['image'], return_tensors='pt')
    inputs['labels'] = batch['label']
    return inputs

Define the custom collation function for batching pixel values and labels.

In [31]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }


Define the function to compute accuracy during evaluation.

In [32]:
accuracy = evaluate.load('accuracy')

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


### **Apply Data Transformations to Dataset**

Apply the defined transformation function to the dataset for preprocessing. </br>
Note: This assumes that data augmentation and normalization have already been handled in the previous pipeline and is ready for fine-tuning.

In [33]:
processed_dataset = dataset.with_transform(transforms)

# **Model Initialization and Trainer Setup**

This section handles the initialization of the model, configuration of training parameters, and setting up the Trainer for fine-tuning, including the datasets, data processing, and evaluation metrics.

### **Initialize Pre-trained Model for Fine-tuning**

Load a pre-trained image classification model, configuring it with the correct label mappings and number of labels for the fine-tuning task.

In [34]:
# Load pre-trained model and configure it for fine-tuning
model = AutoModelForImageClassification.from_pretrained(
    model_path,                  # Path to the pre-trained model
    num_labels=len(labels),      # Set the number of labels for classification
    id2label=id2label,           # Map from ID to label
    label2id=label2id,           # Map from label to ID
    ignore_mismatched_sizes=True # Ignore size mismatches in weights
)

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### **Freeze Model Parameters for Fine-tuning**

Freeze the model parameters except for those related to the classifier layers, allowing fine-tuning only on the specified layers.

In [35]:
# for name, param in model.named_parameters():
#     param.requires_grad = name.startswith(('classifier', 'distillation_classifier'))

for param in model.parameters():
    param.requires_grad = True

We can check how many parameters are there in the model along with how many are actually going to be trained now.

In [36]:
num_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {num_params:,} | Trainable parameters: {trainable_params:,}")

Total parameters: 86,592,776 | Trainable parameters: 86,592,776


### **Define Training Arguments**

Set up the training configuration with parameters such as batch size, number of epochs, learning rate, and logging strategies for the fine-tuning process.

In [37]:
# Define different learning rates
base_lr = 1e-4  # Lower learning rate for base model
classifier_lr = 1e-3  # Higher learning rate for classifier

# Separate model parameters
optimizer_grouped_parameters = [
    {"params": [p for n, p in model.named_parameters() if "classifier" not in n], "lr": base_lr},
    {"params": [p for n, p in model.named_parameters() if "classifier" in n], "lr": classifier_lr},
]

# Define optimizer with different learning rates
optimizer = AdamW(optimizer_grouped_parameters, lr=base_lr)

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [38]:
training_args = TrainingArguments(
    per_device_train_batch_size=64,          # Batch size for training
    per_device_eval_batch_size=64,           # Batch size for evaluation
    eval_strategy="epoch",                   # Evaluation strategy
    save_strategy="epoch",                   # Save strategy
    logging_strategy="epoch",                # Logging strategy
    num_train_epochs=train_epochs,           # Number of training epochs
    learning_rate=base_lr,                   # Learning rate for training
    warmup_ratio=0.1,                        # Warmup ratio for learning rate scheduler
    save_total_limit=5,                      # Limit the number of saved models
    remove_unused_columns=False,             # Retain unused columns in the dataset
    load_best_model_at_end=True,             # Load best model at the end of training
    report_to=['tensorboard'],               # Log to TensorBoard
    logging_dir=logging_dir,                 # Directory for logging
    output_dir=output_dir,                   # Directory for saving outputs
    push_to_hub=True,                        # Push model to Hugging Face Hub
    resume_from_checkpoint=True,             # Resume from checkpoint
)

### **Initialize Trainer**

Initialize the Trainer object with the model, training arguments, data collator, metrics computation, and datasets for training and evaluation.

In [39]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    optimizers=(optimizer, None),
    compute_metrics=compute_metrics,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    tokenizer=processor,
)

<ipython-input-39-d62bfd9f3a74>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# **Model Training and Evaluation**

### **Start Fine-tuning Process**

Initiates the fine-tuning of the model using the Trainer, applying the specified training configurations, such as the batch size, learning rate, and number of epochs. During training, the model will be evaluated at the end of each epoch on the validation dataset using the compute_metrics function, which calculates accuracy.

The model will undergo the following process during fine-tuning:
- **Training**: The model will be trained on the training dataset for the specified number of epochs.
- **Evaluation**: After each epoch, the model will be evaluated on the validation dataset, and accuracy will be computed using the compute_metrics function.
- **Metrics Logging**: The training progress and evaluation results will be logged to TensorBoard and can be monitored during training.

In [40]:
train_results = trainer.train(resume_from_checkpoint=False)

OutOfMemoryError: CUDA out of memory. Tried to allocate 98.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 91.06 MiB is free. Process 31247 has 14.66 GiB memory in use. Of the allocated memory 14.30 GiB is allocated by PyTorch, and 231.76 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### **Save Model and Training State**

After the training process, the model and relevant training state are saved. This includes saving the model weights, training metrics, and the state of the trainer, ensuring that training progress can be restored if needed.

In [ ]:
# Save the trained model
trainer.save_model()

# Log and save training metrics for later reference
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)

# Save the state of the trainer, including configuration and optimizer state
trainer.save_state()